In this project the idea is to work with the Catalonian Mobile Coverage from BigQuery, clean the data and analyze the relationship between te features, see what happens and apply PCA.

Lets load the data from [BigQuery Public Dataset Program](https://cloud.google.com/bigquery/public-data)

To access to the console: https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=catalonian_mobile_coverage_eu&page=dataset&authuser=1&project=primeval-beaker-440708-m7&supportedpurview=project&ws=!1m5!1m4!4m3!1sbigquery-public-data!2scatalonian_mobile_coverage_eu!3smobile_data_2015_2017

In [1]:
# Load libraries
from google.cloud import bigquery
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'google'

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authentication complete')

Read the data from Big Query and store them in a pandas dataframe limited to 100000 lines of reading

In [3]:
project_reader = "orderingandcleaningdata1" # Change this for your project name in GCP
project_public = "bigquery-public-data"
dataset_id = "catalonian_mobile_coverage_eu"
table_id = "mobile_data_2015_2017"
bq_client = bigquery.Client(project_reader)
dataset_ref = bigquery.DatasetReference(project_public, dataset_id)
table_ref = dataset_ref.table(table_id)
table = bq_client.get_table(table_ref)
df = bq_client.list_rows(table,max_results=100000).to_dataframe() # Leemos los primeros 100,000 registros
df.head()

In [4]:
df.info()

In [5]:
df.describe()

In [6]:
# Drop 'downloadSpeed' and 'uploadSpeed' as it has full missing values 
df.drop(['downloadSpeed', 'uploadSpeed'], axis=1, inplace=True)

# Check if the drop has been successful
df

In [7]:
# Let's check the missing values for the feature 'network'
df['network'].isna().sum()

In [8]:
# Check how many times each value appears in the column. There is a big difference between the first and the second value that appear the most, it's almost the double
df['network'].value_counts()

In [9]:
# To fill the values we are going to use the mode, as there is a big difference in the appearance ratio among all the values
df['network'].fillna('Movistar', inplace=True)
df['network'].isna().sum() # Check if there are missing values left

In [10]:
# Repeat with every column that has missing values left like 'operator', 'net', 'postal_code' y 'town_name'
df['operator'].value_counts()

In [11]:
# Same reasoning as before
df['operator'].fillna('Movistar', inplace=True)
df['operator'].isna().sum() # Check that missing values has been replaced succesfully

In [12]:
df['net'].value_counts()

In [13]:
# We fill with the value '4G' following same reasoning as before and nowadays almost everyone has '4G' internet connection
df['net'].fillna('4G', inplace=True)
df['net'].isna().sum() # Revisamos que no queden valores nulos

In [14]:
# Both 'postal_code' and 'town_name' have the same amount of missing values representing the 8,53 % of the total. We have different aproaches, like replacing the values or deleting them.
print(df['postal_code'].isna().sum())
print(df['town_name'].isna().sum())

In [15]:
# In this case I'm going to fill them using the mode and lately I will think if there's is a better approach like deleting the missing data or filling with another value
df['postal_code'].fillna(df['postal_code'].mode()[0], inplace=True)
df['town_name'].fillna(df['town_name'].mode()[0], inplace=True)

# Again check if the filling was completed succesfully
print(df['postal_code'].isna().sum())
print(df['town_name'].isna().sum())

In [16]:
# Now we are gonna check if there is any missing value left in the dataset
df.info()

In [17]:
# Now that we saw there are no missings left let's change the data type of some columns to work better with them

# Let's transform 'object' type columns in 'category' type columns as it uses less memory
for col in ['network', 'operator', 'provider', 'activity', 'description', 'net', 'town_name', 'status']:
    df[col] = df[col].astype('category')

In [18]:
# Convert 'date' and 'hour' column in a datestamp one joining both together
df['date'] = pd.to_datetime(df['date'])
df['hour'] = pd.to_datetime(df['hour'], format='%H:%M:%S').dt.time

df['timestamp'] = df['date'] + pd.to_timedelta(df['hour'].astype(str))

# Drop the columns 'hour' and 'date'
df.drop(['hour', 'date'], axis=1, inplace=True)

In [19]:
# Extract the latitude and longitude values
df['lat'] = df['position_geom'].apply(lambda x: float(x.split('(')[1].split()[0]))
df['lon'] = df['position_geom'].apply(lambda x: float(x.split('(')[1].split()[1].replace(')', '')))

# Drop original column
df.drop(['position_geom'], axis=1, inplace=True)

In [20]:
# Drop dupplicates in case there's any
df.drop_duplicates(inplace=True)
df.info() # 857 were removed so it seems there were duplicates

In [21]:
df.head(20)

In [22]:
# Encode strings into numbers 
def encode_label(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = df[col].astype('category').cat.codes
    return df

In [23]:
cols_to_code = ['network', 'operator', 'description', 'net', 'provider', 'activity', 'town_name']
df_encoded = encode_label(df, cols_to_code)

In [24]:
# Now, all the dataset has been cleaned and encoded to work with it properly in case we use any ML or DL algorithm
df_encoded.head()

In [25]:
# Apply Pearson's correlation method
cor = df_encoded.corr()
cor

In [26]:
# Let's take the 'net' feature' as target and try to see the correlation with the other features
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,10))
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [27]:
# There's no correlation between the features higher than 0.5 and that's what we were looking for
cor_target = abs(cor["net"])
relevant_features = cor_target[cor_target>0.5]
relevant_features

In [28]:
# Let's normalize the data to see if anything changes
from sklearn.preprocessing import MinMaxScaler

cols_to_standard = ['signal', 'speed', 'satellites', 'precission', 'network', 'operator', 'provider', 'activity', 'description', 'net']

min_max_scaler = MinMaxScaler()
df.drop(['timestamp'], axis=1, inplace=True)
df_normalized = pd.DataFrame(min_max_scaler.fit_transform(df_encoded), columns=df.columns)

In [34]:
cor_2 = df_normalized.corr()

In [35]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,10))
sns.heatmap(cor_2, annot=True, cmap=plt.cm.Reds)
plt.show()

#### As we can see there's still no correlation, thats because there's is no 'target' in the dataset so the features are no correlated between them


In [74]:
# Apply PCA and see what happens
from sklearn.decomposition import PCA

pca = PCA()  
pca.fit(df_normalized)

varianza_explicada = pca.explained_variance_ratio_
print(varianza_explicada)

In [76]:
# See the explained variance
varianza_acumulada = np.cumsum(varianza_explicada)
print(varianza_acumulada)

In [78]:
# Plot the elbow graph to see the explained variance
plt.figure(figsize=(8,6))
plt.plot(range(1, len(varianza_acumulada) + 1), varianza_acumulada, marker='o')
plt.xlabel('Número de Componentes')
plt.ylabel('Varianza Acumulada Explicada')
plt.title('Varianza Explicada por Componentes Principales')
plt.grid(True)
plt.show()

In [79]:
# Let's look for the optimal number of features to get the 90% of the explained variance
optimal_components = np.argmax(varianza_acumulada >= 0.90) + 1
print(f"Número óptimo de componentes para retener 90% de la varianza: {optimal_components}")

In [96]:
# We get that we need 7 features, let's now see some graphs
x_pca = pca.transform(df_normalized)

plt.figure(figsize=(8, 6))
plt.scatter(range(len(x_pca[:, 0])), x_pca[:, 0], alpha=0.7, color='blue', label='Primera Componente')
plt.scatter(range(len(x_pca[:, 1])), x_pca[:, 1], alpha=0.7, color='orange', label='Segunda Componente')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title('Proyección de los Datos en las Primeras 2 Componentes')
plt.grid(alpha=0.3)
plt.show()

In [98]:
# Relationship between the 7 first components

# Transformar los datos a las 7 primeras componentes
x_pca_7 = pca.transform(df_normalized)[:, :7]

# Convertir las componentes principales en un DataFrame
pca_df = pd.DataFrame(x_pca_7, columns=[f'PC{i+1}' for i in range(7)])

# Crear un pair plot
sns.pairplot(pca_df, diag_kind='kde', corner=True, height=2)
plt.suptitle('Relación entre las 7 primeras Componentes Principales', y=1.02, fontsize=16)
plt.show()